# Prepare to run in colab

In [1]:
!git clone https://BatyshchevKirill@github.com/BatyshchevKirill/text-detoxification.git

Cloning into 'text-detoxification'...
remote: Enumerating objects: 507, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 507 (delta 2), reused 4 (delta 1), pack-reused 501
Receiving objects: 100% (507/507), 126.30 MiB | 32.36 MiB/s, done.
Resolving deltas: 100% (241/241), done.


In [2]:
import sys
import os

BASE_PATH = "/content/text-detoxification/"
os.chdir(BASE_PATH)
sys.path.append(BASE_PATH)

In [ ]:
# !pip install -r requirements.txt
! pip install -U sentence-transformers
! pip install contractions
! pip install torchmetrics

# Sample test sentences

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv(BASE_PATH + "data/interim/preprocessed_data.csv")
df_src = df['toxic'].sample(n=1000, random_state=42).values
df_tgt = df['detoxed'].sample(n=1000, random_state=42).values

np.savetxt(BASE_PATH + 'data/interim/testing/toxic_test.txt', df_src, fmt="%s", delimiter='\n')
np.savetxt(BASE_PATH + 'data/interim/testing/target_test.txt', df_tgt, fmt="%s", delimiter='\n')

# Run the models on the sample

Run the baseline model

In [5]:
! chmod 777 src/models/predict_model.py
! python src/models/predict_model.py baseline data/interim/testing/toxic_test.txt data/interim/testing/baseline_results.txt --toxic_words_path data/interim/toxic_words.txt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100% 1000/1000 [03:36<00:00,  4.61it/s]
Results saved to data/interim/testing/baseline_results.txt


Run the pretrained T5 model

In [6]:
! python src/models/predict_model.py t5 data/interim/testing/toxic_test.txt data/interim/testing/t5_results.txt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100% 1000/1000 [11:39<00:00,  1.43it/s]
Results saved to data/interim/testing/t5_results.txt


Run my transformer model

In [ ]:
pass

# Evaluate the models

In [7]:
from src.models.evaluate import bleu, rouge, ToxicityClassifier, SemanticSimilarityClassifier

tox_class = ToxicityClassifier()
sem_sim_class = SemanticSimilarityClassifier()

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load the data

In [9]:
with open("data/interim/testing/baseline_results.txt", "r") as f:
    baseline_data = f.read().split("\n")

with open("data/interim/testing/transformer_results.txt", "r") as f:
    transformer_data = f.read().split("\n")

with open("data/interim/testing/t5_results.txt", "r") as f:
    t5_data = f.read().split("\n")

with open("data/interim/testing/toxic_test.txt", "r") as f:
    source_data = f.read().split("\n")

with open("data/interim/testing/target_test.txt", "r") as f:
    target_data = f.read().split("\n")

Evaluate the models using the BLEU score

In [14]:
baseline_bleu = [float(bleu(item)) for item in zip(baseline_data, target_data)]
# transformer_bleu = [float(bleu(item)) for item in zip(transformer_data, target_data)]
t5_bleu = [float(bleu(item)) for item in zip(t5_data, target_data)]


print("Average BLEU score of the baseline model (zeros excluded) is:")
print(sum(baseline_bleu) / (len(baseline_bleu) - baseline_bleu.count(0.0)))
print()

# print("Average BLEU score of the transformer model (zeros excluded) is:")
# print(sum(transformer_bleu) / sum(transformer_bleu != 0))
# print()

print("Average BLEU score of the pretrained T5 model (zeros excluded) is:")
print(sum(t5_bleu) / (len(t5_bleu) - t5_bleu.count(0.0)))

Average BLEU score of the baseline model (zeros excluded) is:
0.3615299778363921

Average BLEU score of the pretrained T5 model (zeros excluded) is:
0.4156749470582331


Calculate the rouge scores

In [16]:
baseline_rouge = np.array(
    [rouge(item) for item in zip(baseline_data, target_data)]
).T
# transformer_rouge = np.array(
#     [rouge(item) for item in zip(transformer_data, target_data)]
# ).T
t5_rouge = np.array(
    [rouge(item) for item in zip(t5_data, target_data)]
 ).T



print("Average ROUGE F1 score of the baseline model is [ROUGE1 F1, ROUGE2 F1]:")
print(baseline_rouge.sum(axis=1) / baseline_rouge.shape[1])
print()

# print("Average ROUGE F1 score of the transformer model is [ROUGE1 F1, ROUGE2 F1]:")
# print(transformer_rouge.sum(axis=1) / transformer_rouge.shape[1])
# print()

print("Average ROUGE F1 score of the pretrained T5 model is [ROUGE1 F1, ROUGE2 F1]:")
print(t5_rouge.sum(axis=1) / t5_rouge.shape[1])

Average ROUGE F1 score of the baseline model is [ROUGE1 F1, ROUGE2 F1]:
[0.57221425 0.33180147]

Average ROUGE F1 score of the pretrained T5 model is [ROUGE1 F1, ROUGE2 F1]:
[0.55578184 0.30229768]


Now let us measure the toxicity of the generated data

In [17]:
from tqdm import tqdm

baseline_tox = [float(tox_class(item)) for item in tqdm(baseline_data)]
# transformer_tox = [float(tox_class(item)) for item in tqdm(transformer_data)]
t5_tox = [float(tox_class(item)) for item in tqdm(t5_data)]

print("Average toxicity score of the baseline model is:")
print(sum(baseline_tox) / len(baseline_tox))
print()

# print("Average toxicity score of the transformer model is:")
# print(sum(transformer_tox) / len(transformer_tox))
# print()

print("Average toxicity score of the T5 model is:")
print(sum(t5_tox) / len(t5_tox))

100%|██████████| 1000/1000 [01:35<00:00, 10.50it/s]

Average toxicity score of the baseline model is:
0.3153660131049328

Average toxicity score of the T5 model is:
0.3388119374278649


Now we measure the semantic similarity between the produced and original texts

In [20]:
baseline_sim = [
    float(sem_sim_class(item))
    for item in tqdm(zip(baseline_data, source_data), total=len(source_data))
    ]
# transformer_sim = [
#     float(sem_sim_class(item))
#     for item in tqdm(zip(transformer_data, source_data), total=len(source_data))
#     ]
t5_sim = [
    float(sem_sim_class(item))
    for item in tqdm(zip(t5_data, source_data), total=len(source_data))
    ]


print("Average semantic similarity between the baseline model predictions and source texts:")
print(sum(baseline_sim) / len(baseline_sim))
print()

# print("Average semantic similarity between the transformer model predictions and source texts:")
# print(sum(transformer_sim) / len(transformer_sim))
# print()

print("Average semantic similarity between the Pretrained T5 model predictions and source texts:")
print(sum(t5_sim) / len(t5_sim))

100%|█████████▉| 1000/1001 [00:24<00:00, 41.65it/s]Average semantic similarity between the baseline model predictions and source texts:
0.8724988852441311

Average semantic similarity between the Pretrained T5 model predictions and source texts:
0.8535619608014822

